# Header

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import neccesary packages
import numpy as np
import random

import torch
from torch import nn, optim
from torchvision import datasets, transforms, models

In [3]:
# Setup for getting the reproduciblility results
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# Check whether we are using GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Load the dataset

In [5]:
data_dir = '/content/drive/MyDrive/Colab1/Pytorch/flower_data'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'
valid_dir = data_dir + '/valid'

In [6]:
# Define trainformation
trans = transforms.Compose([transforms.Resize(256),
                           transforms.CenterCrop(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406],
                                                [0.229, 0.224, 0.225])])

In [7]:
# Load the datasets with ImageFolder
training_set = datasets.ImageFolder(train_dir, transform=trans)
testing_set = datasets.ImageFolder(test_dir, transform=trans)
validation_set = datasets.ImageFolder(valid_dir, transform=trans)

In [8]:
# Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(training_set, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testing_set, batch_size=32)
validate_loader = torch.utils.data.DataLoader(validation_set, batch_size=32)

# Define Functions

In [9]:
# Function for validation model
def validation(model, validateloader, criterion):

  val_loss = 0
  accuracy = 0

  for images, labels in iter(validateloader):

    images, labels = images.to('cuda'), labels.to('cuda')

    output = model.forward(images)
    val_loss += criterion(output, labels).item()

    probabilities = torch.exp(output)

    equality = (labels.data == probabilities.max(dim=1)[1])
    accuracy += equality.type(torch.FloatTensor).mean()

  return val_loss, accuracy

In [10]:
# Function for training model
import time 
def train_model():
  max_epoch = 5
  print_every = 20
  n_steps = 0
  model.to('cuda')
  
  start_train = time.time()
  for n_epochs in range(max_epoch):
    print(f' - Epoch {n_epochs + 1}')
    start_epoch = time.time()
    model.train()
    
    epoch_loss = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
      images, labels = images.to('cuda'), labels.to('cuda')
      
      optimizer.zero_grad()
      
      output = model.forward(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()
      
      epoch_loss += loss.item()
      if n_steps % print_every == 0:
        model.eval()
        # Turn off gradients for validation, saves memory and computation
        with torch.no_grad():
          validation_loss, accuracy = validation(model, validate_loader, criterion)
          
        print(f"Batch: {batch_idx}/{len(train_loader)}.. ",
              #f"Training Loss: {running_loss/print_every: 3f}.. ",
              f"Training Loss: {epoch_loss/print_every: 3f}.. ",
              f"Validation Loss: {validation_loss/len(validate_loader):.3f}..", 
              f"Validation Accuracy: {accuracy/len(validate_loader):.3f}")
        
        epoch_loss = 0
        model.train()
      n_steps += 1
    end_epoch = time.time()
    print(f'--> Epoch {n_epochs + 1} costs {end_epoch - start_epoch} seconds')
  end_train = time.time()
  print(f'Training time: {end_train - start_train}')

In [11]:
# Save model for using when we want to use in the later
def save_checkpoint(model):

  model.class_to_idx = training_set.class_to_idx

  checkpoint = {'arch': "vgg16",
                'class_to_idx': model.class_to_idx,
                'model_state_dict': model.state_dict()
                }
  torch.save(checkpoint, '/content/drive/MyDrive/Colab1/Pytorch/checkpoint.pth')

In [12]:
# Function for test model
def test_model(model, test_loader):
  # Do validation on the test set
  model.eval()
  model.to('cuda')

  with torch.no_grad():
    accuracy = 0
    for images, labels in iter(test_loader):
      images, labels = images.to('cuda'), labels.to('cuda')
      output = model.forward(images)
      probabilities = torch.exp(output)
      equality = (labels.data == probabilities.max(dim=1)[1])
      accuracy += equality.type(torch.FloatTensor).mean()
    print("Test Accuracy: {}".format(accuracy/len(test_loader)))

# **The first approach**
**Load VGG-16 model (no using pre-trained model - train from scratch)**


## Load model

In [13]:
model = models.vgg16(pretrained=False)

In [14]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

- The output of the 'classifier' is 1 of out 1000 labels, but the number of categories in dataset is 102 ➡ customize the classifier 

In [15]:
# Build custom classifier
classifier = nn.Sequential(nn.Linear(25088, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout (p=0.5, inplace=False),
                           nn.Linear(4096, 102),
                           nn.LogSoftmax(dim=1)
                          )

In [16]:
model.classifier = classifier

In [17]:
print (model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Define 'loss function' and 'optimizer'

In [18]:
# 1st approach
criterion = nn.NLLLoss() # Negative log likelihood
optimizer = optim.Adam(model.parameters(), lr=0.001) # Learning rate

## Train model

In [19]:
train_model()

 - Epoch 1
Batch: 0/103..  Training Loss:  0.231504..  Validation Loss: 77.748.. Validation Accuracy: 0.022
Batch: 20/103..  Training Loss:  8.653890..  Validation Loss: 4.545.. Validation Accuracy: 0.025
Batch: 40/103..  Training Loss:  4.546487..  Validation Loss: 4.514.. Validation Accuracy: 0.023
Batch: 60/103..  Training Loss:  4.537993..  Validation Loss: 4.511.. Validation Accuracy: 0.034
Batch: 80/103..  Training Loss:  4.519862..  Validation Loss: 4.505.. Validation Accuracy: 0.034
Batch: 100/103..  Training Loss:  4.543632..  Validation Loss: 4.513.. Validation Accuracy: 0.034
--> Epoch 1 costs 4659.005563497543 seconds
 - Epoch 2
Batch: 17/103..  Training Loss:  4.063340..  Validation Loss: 4.519.. Validation Accuracy: 0.025
Batch: 37/103..  Training Loss:  4.519714..  Validation Loss: 4.518.. Validation Accuracy: 0.025
Batch: 57/103..  Training Loss:  4.514330..  Validation Loss: 4.512.. Validation Accuracy: 0.025
Batch: 77/103..  Training Loss:  4.522397..  Validation Loss

## Save model

In [20]:
save_checkpoint(model)

## Test model

In [21]:
test_model(model, test_loader)

Test Accuracy: 0.028846153989434242


# **The second approach**
**Load VGG-16 model (using pre-trained)**

## Load model

In [22]:
model = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [23]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

- The output of the 'classifier' is 1 of out 1000 labels, but the number of categories in dataset is 102 ➡ customize the classifier 

In [24]:
#If pretrained = True (2nd approach)
#Freeze pretrained model parameters to avoid backpropogating through them
for parameter in model.parameters():
  parameter.requires_grad = False

In [25]:
# Build custom classifier
classifier = nn.Sequential(nn.Linear(25088, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout(p=0.5, inplace=False),
                           nn.Linear(4096, 4096),
                           nn.ReLU(inplace=True),
                           nn.Dropout (p=0.5, inplace=False),
                           nn.Linear(4096, 102),
                           nn.LogSoftmax(dim=1)
                          )

In [26]:
model.classifier = classifier

In [27]:
print (model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Define 'loss function' and 'optimizer'

In [28]:
# 2nd approach
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

## Train model

In [29]:
train_model()

 - Epoch 1
Batch: 0/103..  Training Loss:  0.231763..  Validation Loss: 10.570.. Validation Accuracy: 0.011
Batch: 20/103..  Training Loss:  5.203010..  Validation Loss: 3.439.. Validation Accuracy: 0.243
Batch: 40/103..  Training Loss:  3.062568..  Validation Loss: 2.138.. Validation Accuracy: 0.523
Batch: 60/103..  Training Loss:  2.107105..  Validation Loss: 1.433.. Validation Accuracy: 0.622
Batch: 80/103..  Training Loss:  1.905939..  Validation Loss: 1.277.. Validation Accuracy: 0.660
Batch: 100/103..  Training Loss:  1.824228..  Validation Loss: 0.968.. Validation Accuracy: 0.767
--> Epoch 1 costs 395.72886395454407 seconds
 - Epoch 2
Batch: 17/103..  Training Loss:  1.131150..  Validation Loss: 0.974.. Validation Accuracy: 0.735
Batch: 37/103..  Training Loss:  1.327264..  Validation Loss: 0.900.. Validation Accuracy: 0.780
Batch: 57/103..  Training Loss:  1.267614..  Validation Loss: 0.931.. Validation Accuracy: 0.739
Batch: 77/103..  Training Loss:  1.168633..  Validation Los

## Save model

In [30]:
save_checkpoint(model)

## Test model

In [31]:
test_model(model, test_loader)

Test Accuracy: 0.7694838047027588
